In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.parent.resolve()))

from pprint import PrettyPrinter
pp = PrettyPrinter()

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\n\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [2]:
from trulens_eval.keys import *

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


In [3]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
# response = query_engine.query("What did the author do growing up?")
# print(response)

In [4]:
# For aggregation,
import numpy as np

from trulens_eval import feedback, Feedback, Query, Tru, TruLlama

In [5]:
# Construct feedback functions.

hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on_input_output()
# By default this will evaluate feedback on main app input and main app output.

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()
# Same default inputs as the above.

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    TruLlama.select_source_nodes().node.text
).aggregate(np.min)
# First feedback arg is the main app input while the second is the context
# chunks retrieved from the main app output, output of `query`.

feedbacks = [
#    f_lang_match, 
#    f_qa_relevance, 
#    f_qs_relevance
]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface api: 0requests [00:00, ?requests/s]

openai api: 0requests [00:00, ?requests/s]

✅ In language_match, input text1 will be set to *.__record__.main_input or `Query.RecordInput` .
✅ In language_match, input text2 will be set to *.__record__.main_output or `Query.RecordOutput` .
✅ In relevance, input prompt will be set to *.__record__.main_input or `Query.RecordInput` .
✅ In relevance, input response will be set to *.__record__.main_output or `Query.RecordOutput` .
✅ In qs_relevance, input question will be set to *.__record__.main_input or `Query.RecordInput` .
✅ In qs_relevance, input statement will be set to *.__record__.app.query.rets.source_nodes[:].node.text .


In [6]:
l = TruLlama(app=query_engine, feedbacks=feedbacks)

✅ app app_hash_6e1ec7f7e22a6bb5b127530aec670f8b -> default.sqlite


In [7]:
# Show which components of the llama index app have been instrumented (will be
# tracked as components in the dashboard).

l.print_instrumented()

Other component: *.__app__.app
Other component: *.__app__.app._response_synthesizer
Other component: *.__app__.app._response_synthesizer._response_builder
Prompt component: *.__app__.app._response_synthesizer._response_builder._refine_template
Prompt component: *.__app__.app._response_synthesizer._response_builder._refine_template.prompt
Other component: *.__app__.app._response_synthesizer._response_builder.service_context
Other component: *.__app__.app._response_synthesizer._response_builder.service_context.embed_model
Other component: *.__app__.app._response_synthesizer._response_builder.service_context.llm_predictor
LLM component: *.__app__.app._response_synthesizer._response_builder.service_context.llm_predictor.llm
Other component: *.__app__.app._response_synthesizer._response_builder.service_context.node_parser
Other component: *.__app__.app._response_synthesizer._response_builder.service_context.prompt_helper
Prompt component: *.__app__.app._response_synthesizer._response_builde

In [8]:
res, record = l.query_with_record("Who is Shayak?")

Token indices sequence length is longer than the specified maximum sequence length for this model (1818 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
# Start the dashboard here:
# proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent)

# If using deferred feedback evaluation, need to start this too:
# thread = Tru().start_evaluator(restart=True)

✅ record record_hash_e463aa0f533d752b586a31c7b8385d34 from app_hash_6e1ec7f7e22a6bb5b127530aec670f8b -> default.sqlite


In [10]:
f = f_qs_relevance.run(record=record, app=l)

In [11]:
f.dict()

{'feedback_result_id': 'feedback_result_hash_00b769b19601e220a854aacd7a6f8257',
 'record_id': 'record_hash_e463aa0f533d752b586a31c7b8385d34',
 'feedback_definition_id': 'feedback_definition_hash_ca90883398099199aac8facf66f0aaae',
 'last_ts': datetime.datetime(2023, 6, 20, 14, 19, 4, 536474),
 'status': <FeedbackResultStatus.DONE: 'done'>,
 'cost': {'n_tokens': 0, 'cost': 0.0},
 'tags': '',
 'name': 'qs_relevance',
 'calls': [{'args': {'question': 'Who is Shayak?',
    'statement': "of my attention. It had already eaten Arc, and was in the process of eating essays too. Either YC was my life's work or I'd have to leave eventually. And it wasn't, so I would.\n\nIn the summer of 2012 my mother had a stroke, and the cause turned out to be a blood clot caused by colon cancer. The stroke destroyed her balance, and she was put in a nursing home, but she really wanted to get out of it and back to her house, and my sister and I were determined to help her do it. I used to fly up to Oregon to vis